<a href="https://colab.research.google.com/github/TimHBSWFL/UCSD-ML-Capstone/blob/main/Evaluating_CLIP_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install transformers pillow torch

In [1]:
from transformers import CLIPModel, CLIPProcessor
import torch
from PIL import Image
import os
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.listdir('/content/drive/My Drive/Capstone Data Collection/models')

['yelp_clip_vit_fine_dining_lr5e-5_batch32_2024_09_29_v1.pth',
 'yelp_clip_vit_fine_dining_lr5e-5_batch32_2024_09_30_v1.pth']

In [4]:
model_path = "/content/drive/My Drive/Capstone Data Collection/models/yelp_clip_vit_fine_dining_lr5e-5_batch32_2024_09_30_v1.pth"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

state_dict = torch.load(model_path, map_location="cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(state_dict)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

<ipython-input-4-215102d60a8f>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location="cuda" if torch.cuda.is_available() else "cp

<All keys matched successfully>

In [5]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [6]:
labels = ["fast food", "fine dining"]

text_inputs = processor(text=labels, return_tensors="pt", padding=True).to(device)

test_images_dir = "/content/drive/My Drive/Capstone Data Collection/test"
fine_dining_output_dir = "/content/drive/My Drive/Capstone Data Collection/predictions/fine dining"
fast_food_output_dir = "/content/drive/My Drive/Capstone Data Collection/predictions/fast food"

os.makedirs(fast_food_output_dir, exist_ok=True)
os.makedirs(fine_dining_output_dir, exist_ok=True)

model.eval()


for image_filename in os.listdir(test_images_dir):
    if image_filename.endswith(".jpg"):
        image_path = os.path.join(test_images_dir, image_filename)
        image = Image.open(image_path).convert("RGB")

        image_inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            image_features = model.get_image_features(**image_inputs)
            text_features = model.get_text_features(**text_inputs)
            logits_per_image = (image_features @ text_features.T).softmax(dim=-1)

        predicted_label_idx = logits_per_image.argmax().item()
        predicted_label = labels[predicted_label_idx]

        print(f"Image: {image_filename}, Predicted label: {predicted_label}")

        if predicted_label == "fast food":
            shutil.copy(image_path, os.path.join(fast_food_output_dir, image_filename))
        else:
            shutil.copy(image_path, os.path.join(fine_dining_output_dir, image_filename))

print("Classification complete!")

Image: -_8iUXhThD_VobTagop6AA.jpg, Predicted label: fine dining
Image: -_82b4r1oFF68VGzdG5kAw.jpg, Predicted label: fast food
Image: -__4fB3-t0HUSOHb0lHZGA.jpg, Predicted label: fine dining
Image: -__DdEVsZPQkwqHgAoZOOg.jpg, Predicted label: fine dining
Image: _1nf7andUukJoMAtR4E-_w.jpg, Predicted label: fine dining
Image: _4p6uGL7he9evFOfnfX0xQ.jpg, Predicted label: fine dining
Image: _5A2laRLJ2fEzjYrGEL9Og.jpg, Predicted label: fast food
Image: _4HtdlrzD9WsMoRdje1Y-g.jpg, Predicted label: fast food
Image: _1iTcUhYsR6v4lu5P_CzkA.jpg, Predicted label: fine dining
Image: _8Bbj7FNHbKg8lZn7_XofA.jpg, Predicted label: fine dining
Image: _6GUQWHJjVAgwLqOmFHCew.jpg, Predicted label: fine dining
Image: _06w5Yq2UXOTr7NLnxG-xA.jpg, Predicted label: fine dining
Image: _0dvZHaTGJWJNpRD5LOkDw.jpg, Predicted label: fine dining
Image: _68by-21aE3RDSBNG2rQCw.jpg, Predicted label: fine dining
Image: _59_To_iPgaMOwpy453eDA.jpg, Predicted label: fine dining
Image: _24IPvpDRoOpnJTORBZ9fA.jpg, Predicted l